In [ ]:
import json
import hashlib

# Helper function for hashing passwords
def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

# Transaction Class
class Transaction:
    def __init__(self, transaction_id, type, amount, balance):
        self.transaction_id = transaction_id
        self.type = type
        self.amount = amount
        self.balance = balance

# Account Class
class Account:
    def __init__(self, account_number, holder_name, password):
        self.account_number = account_number
        self.holder_name = holder_name
        self.password_hash = hash_password(password)
        self.balance = 0.0
        self.transactions = []

    def authenticate(self, password):
        return self.password_hash == hash_password(password)

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")
        self.balance += amount
        transaction = Transaction(len(self.transactions) + 1, "Deposit", amount, self.balance)
        self.transactions.append(transaction)

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if amount > self.balance:
            raise ValueError("Insufficient funds.")
        self.balance -= amount
        transaction = Transaction(len(self.transactions) + 1, "Withdrawal", amount, self.balance)
        self.transactions.append(transaction)

    def get_transaction_history(self):
        return [vars(t) for t in self.transactions]

# Bank Class
class Bank:
    def __init__(self):
        self.accounts = {}

    def create_account(self, account_number, holder_name, password):
        if account_number in self.accounts:
            raise ValueError("Account number already exists.")
        account = Account(account_number, holder_name, password)
        self.accounts[account_number] = account

    def get_account(self, account_number, password):
        account = self.accounts.get(account_number)
        if not account or not account.authenticate(password):
            raise ValueError("Authentication failed.")
        return account

    def save_data(self, filename="bank_data.json"):
        data = {
            account_number: {
                "holder_name": account.holder_name,
                "password_hash": account.password_hash,
                "balance": account.balance,
                "transactions": account.get_transaction_history()
            } for account_number, account in self.accounts.items()
        }
        with open(filename, "w") as file:
            json.dump(data, file, indent=4)

    def load_data(self, filename="bank_data.json"):
        try:
            with open(filename, "r") as file:
                data = json.load(file)
                for account_number, account_data in data.items():
                    account = Account(
                        account_number,
                        account_data["holder_name"],
                        "temporary_password"
                    )
                    account.password_hash = account_data["password_hash"]
                    account.balance = account_data["balance"]
                    for transaction_data in account_data["transactions"]:
                        transaction = Transaction(
                            transaction_data["transaction_id"],
                            transaction_data["type"],
                            transaction_data["amount"],
                            transaction_data["balance"]
                        )
                        account.transactions.append(transaction)
                    self.accounts[account_number] = account
        except FileNotFoundError:
            pass

# Example Usage
if __name__ == "__main__":
    bank = Bank()
    bank.load_data()

    while True:
        print("\n--- Banking System ---")
        print("1. Create Account")
        print("2. Deposit")
        print("3. Withdraw")
        print("4. View Balance")
        print("5. Transaction History")
        print("6. Exit")

        choice = input("Enter your choice: ")
        try:
            if choice == "1":
                account_number = input("Enter account number: ")
                holder_name = input("Enter holder name: ")
                password = input("Enter password: ")
                bank.create_account(account_number, holder_name, password)
                print("Account created successfully!")

            elif choice == "2":
                account_number = input("Enter account number: ")
                password = input("Enter password: ")
                amount = float(input("Enter deposit amount: "))
                account = bank.get_account(account_number, password)
                account.deposit(amount)
                print(f"Deposited {amount}. New balance: {account.balance}")

            elif choice == "3":
                account_number = input("Enter account number: ")
                password = input("Enter password: ")
                amount = float(input("Enter withdrawal amount: "))
                account = bank.get_account(account_number, password)
                account.withdraw(amount)
                print(f"Withdrawn {amount}. New balance: {account.balance}")

            elif choice == "4":
                account_number = input("Enter account number: ")
                password = input("Enter password: ")
                account = bank.get_account(account_number, password)
                print(f"Current balance: {account.balance}")

            elif choice == "5":
                account_number = input("Enter account number: ")
                password = input("Enter password: ")
                account = bank.get_account(account_number, password)
                history = account.get_transaction_history()
                print("Transaction History:")
               for t in history:
                    print(t)

            elif choice == "6":
                bank.save_data()
                print("Data saved. Exiting...")
                break

            else:
                print("Invalid choice. Please try again.")
        except Exception as e:
            print(f"Error: {e}")



--- Banking System ---
1. Create Account
2. Deposit
3. Withdraw
4. View Balance
5. Transaction History
6. Exit
Enter your choice: 1


In [1]:
import sqlite3
import tkinter as tk
from tkinter import messagebox, ttk

# Database setup
conn = sqlite3.connect('inventory_management.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS Product (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    reorder_point INTEGER NOT NULL,
    supplier_id INTEGER,
    FOREIGN KEY (supplier_id) REFERENCES Supplier(id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Supplier (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    contact_info TEXT
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS OrderHistory (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_id INTEGER NOT NULL,
    supplier_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (product_id) REFERENCES Product(id),
    FOREIGN KEY (supplier_id) REFERENCES Supplier(id)
)''')

conn.commit()

# Classes
class Product:
    def __init__(self, name, quantity, reorder_point, supplier_id=None):
        self.name = name
        self.quantity = quantity
        self.reorder_point = reorder_point
        self.supplier_id = supplier_id

    def save(self):
        cursor.execute('''INSERT INTO Product (name, quantity, reorder_point, supplier_id) VALUES (?, ?, ?, ?)''',
                       (self.name, self.quantity, self.reorder_point, self.supplier_id))
        conn.commit()

class Supplier:
    def __init__(self, name, contact_info):
        self.name = name
        self.contact_info = contact_info

    def save(self):
        cursor.execute('''INSERT INTO Supplier (name, contact_info) VALUES (?, ?)''',
                       (self.name, self.contact_info))
        conn.commit()

# Inventory Management
class Inventory:
    @staticmethod
    def add_product(name, quantity, reorder_point, supplier_id=None):
        product = Product(name, quantity, reorder_point, supplier_id)
        product.save()

    @staticmethod
    def update_product(product_id, name=None, quantity=None, reorder_point=None):
        updates = []
        params = []

        if name:
            updates.append("name = ?")
            params.append(name)
        if quantity is not None:
            updates.append("quantity = ?")
            params.append(quantity)
        if reorder_point is not None:
            updates.append("reorder_point = ?")
            params.append(reorder_point)

        params.append(product_id)

        cursor.execute(f"UPDATE Product SET {', '.join(updates)} WHERE id = ?", params)
        conn.commit()

    @staticmethod
    def delete_product(product_id):
        cursor.execute('''DELETE FROM Product WHERE id = ?''', (product_id,))
        conn.commit()

    @staticmethod
    def view_inventory():
        cursor.execute('''SELECT * FROM Product''')
        return cursor.fetchall()

# Reporting
class Reports:
    @staticmethod
    def inventory_report():
        return Inventory.view_inventory()

    @staticmethod
    def supplier_performance():
        cursor.execute('''SELECT s.name, COUNT(o.id) AS orders_count
                          FROM Supplier s
                          LEFT JOIN OrderHistory o ON s.id = o.supplier_id
                          GROUP BY s.id''')
        return cursor.fetchall()

# GUI Application
class InventoryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Inventory Management System")

        # Frames
        self.frame = tk.Frame(root)
        self.frame.pack(pady=20, padx=20)

        self.tree = ttk.Treeview(root, columns=("ID", "Name", "Quantity", "Reorder Point", "Supplier ID"), show="headings")
        self.tree.heading("ID", text="ID")
        self.tree.heading("Name", text="Name")
        self.tree.heading("Quantity", text="Quantity")
        self.tree.heading("Reorder Point", text="Reorder Point")
        self.tree.heading("Supplier ID", text="Supplier ID")
        self.tree.pack(pady=20)

        # Buttons
        tk.Button(self.frame, text="Add Product", command=self.add_product).grid(row=0, column=0, padx=10)
        tk.Button(self.frame, text="Update Product", command=self.update_product).grid(row=0, column=1, padx=10)
        tk.Button(self.frame, text="Delete Product", command=self.delete_product).grid(row=0, column=2, padx=10)
        tk.Button(self.frame, text="View Inventory", command=self.view_inventory).grid(row=0, column=3, padx=10)

    def add_product(self):
        def save():
            name = name_entry.get()
            quantity = int(quantity_entry.get())
            reorder_point = int(reorder_point_entry.get())
            supplier_id = int(supplier_id_entry.get()) if supplier_id_entry.get() else None
            Inventory.add_product(name, quantity, reorder_point, supplier_id)
            messagebox.showinfo("Success", "Product added successfully!")
            add_window.destroy()

        add_window = tk.Toplevel(self.root)
        add_window.title("Add Product")

        tk.Label(add_window, text="Name").grid(row=0, column=0, padx=10, pady=5)
        name_entry = tk.Entry(add_window)
        name_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(add_window, text="Quantity").grid(row=1, column=0, padx=10, pady=5)
        quantity_entry = tk.Entry(add_window)
        quantity_entry.grid(row=1, column=1, padx=10, pady=5)

        tk.Label(add_window, text="Reorder Point").grid(row=2, column=0, padx=10, pady=5)
        reorder_point_entry = tk.Entry(add_window)
        reorder_point_entry.grid(row=2, column=1, padx=10, pady=5)

        tk.Label(add_window, text="Supplier ID").grid(row=3, column=0, padx=10, pady=5)
        supplier_id_entry = tk.Entry(add_window)
        supplier_id_entry.grid(row=3, column=1, padx=10, pady=5)

        tk.Button(add_window, text="Save", command=save).grid(row=4, column=0, columnspan=2, pady=10)

    def update_product(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showerror("Error", "Please select a product to update.")
            return

        product_id = self.tree.item(selected[0], "values")[0]

        def save():
            name = name_entry.get()
            quantity = int(quantity_entry.get()) if quantity_entry.get() else None
            reorder_point = int(reorder_point_entry.get()) if reorder_point_entry.get() else None
            Inventory.update_product(product_id, name=name, quantity=quantity, reorder_point=reorder_point)
            messagebox.showinfo("Success", "Product updated successfully!")
            update_window.destroy()

        update_window = tk.Toplevel(self.root)
        update_window.title("Update Product")

        tk.Label(update_window, text="Name").grid(row=0, column=0, padx=10, pady=5)
        name_entry = tk.Entry(update_window)
        name_entry.grid(row=0, column=1, padx=10, pady=5)

        tk.Label(update_window, text="Quantity").grid(row=1, column=0, padx=10, pady=5)
        quantity_entry = tk.Entry(update_window)
        quantity_entry.grid(row=1, column=1, padx=10, pady=5)

        tk.Label(update_window, text="Reorder Point").grid(row=2, column=0, padx=10, pady=5)
        reorder_point_entry = tk.Entry(update_window)
        reorder_point_entry.grid(row=2, column=1, padx=10, pady=5)

        tk.Button(update_window, text="Save", command=save).grid(row=3, column=0, columnspan=2, pady=10)

    def delete_product(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showerror("Error", "Please select a product to delete.")
            return

        product_id = self.tree.item(selected[0], "values")[0]
        Inventory.delete_product(product_id)
        messagebox.showinfo("Success", "Product deleted successfully!")

    def view_inventory(self):
        for i in self.tree.get_children():
            self.tree.delete(i)

        inventory = Inventory.view_inventory()
        for item in inventory:
            self.tree.insert("", "end", values=item)

if __name__ == "__main__":
    root = tk.Tk()
    app = InventoryApp(root)
    root.mainloop()

conn.close()
